In [8]:
import pandas as pd
import redcap
import requests
import io
import sys
from credentials import token_pid353
from credentials import redcap_api_url
sys.path.insert(0, '../toolbox/')
from general_tools import compare_similar_dataframes

In [9]:
def redcap_API_export(url,token):
    """
    all data API request
    """

    fields = {
        'token': token,
        'content': 'record',
        'format': 'csv',
        'type': 'flat'}

    r = requests.post(url, data=fields)
    
    df = pd.read_csv(io.StringIO(r.content.decode("utf-8")), index_col=0)

    return df

def wide_to_long(df_pid171):
    """
    REDCap PID171 is in wide format with unique sample ID's and PCR_target of n1 and n2 containing fields sars_cov2_below_lod (n1 and n2) and sars_cov2_avg_conc (n1 and n2).

    Transform long format: repeat sample ID's for n1 target and n2 target (PCR_target field). Single column of sars_cov2_below_lod and sars_cov2_avg_conc.

    """
    df_pid171 = df_pid171.reset_index().copy()

    #identify columns to melt, and all the rest
    melt_clms = ['n1_sars_cov2_avg_conc', 'n2_sars_cov2_avg_conc', 'n1_sars_cov2_below_lod', 'n2_sars_cov2_below_lod']
    not_melt_clms = df_pid171.columns[~df_pid171.columns.isin(melt_clms)]

    #perform melt for avg_conc and keep all other columns
    df_melt_conc = pd.melt(df_pid171, value_vars = ['n1_sars_cov2_avg_conc', 'n2_sars_cov2_avg_conc'], var_name = "pcr_target", value_name = 'sars_cov2_avg_conc', id_vars = not_melt_clms )
    #perform melt for below_lod and only keep the value column (below_lod)
    df_melt_lod = pd.melt(df_pid171, value_vars = ['n1_sars_cov2_below_lod', 'n2_sars_cov2_below_lod'], var_name = "pcr_target", value_name = 'sars_cov2_below_lod', id_vars = ["sample_id"] )

    #change the PCR_target column to only first 2 letters (n1 or n2)
    df_melt_lod["pcr_target"] = df_melt_lod["pcr_target"].str[0:2]
    df_melt_conc["pcr_target"] = df_melt_lod["pcr_target"].str[0:2]

    #merge the dataframes together
    df_pid171 = pd.merge(df_melt_conc, df_melt_lod, how = "inner", left_on = ["sample_id", "pcr_target"], right_on = ["sample_id", "pcr_target"])

    return df_pid171


### Version1 of Wide-to-long transform:

In [10]:
df_pid171 = redcap_API_export('https://redcap.doh.wa.gov/api/', "AB21CE90EF475E08AC11F92105A39690")

df_pid171 = wide_to_long(df_pid171)

In [11]:
df_pid171.shape

(1714, 97)

### Version2 of Wide-to-long transform:

In [12]:
token_pid171 = "AB21CE90EF475E08AC11F92105A39690"
redcap_api_url = 'https://redcap.doh.wa.gov/api/'
project = redcap.Project(url = redcap_api_url, token = token_pid171)
df_pid171_v2 = project.export_records(format_type = "df")

In [13]:

df_pid171_v2 = df_pid171_v2.reset_index().copy()

df_conc_melt = df_pid171_v2.melt(id_vars = "sample_id", value_vars = ['n1_sars_cov2_avg_conc', 'n2_sars_cov2_avg_conc'], var_name = "pcr_target", value_name = 'sars_cov2_avg_conc')
df_lod_melt = df_pid171_v2.melt(id_vars = "sample_id", value_vars = [ 'n1_sars_cov2_below_lod', 'n2_sars_cov2_below_lod'], var_name = "pcr_target", value_name = 'sars_cov2_below_lod' )

df_conc_melt["pcr_target"] = df_conc_melt["pcr_target"].str[0:2]
df_lod_melt["pcr_target"] = df_lod_melt["pcr_target"].str[0:2]


df_melt = pd.merge(df_conc_melt, df_lod_melt, left_on = ["sample_id", "pcr_target"],
                                             right_on = ["sample_id", "pcr_target"],
                                             how = "inner")


melt_clms = ['n1_sars_cov2_avg_conc', 'n2_sars_cov2_avg_conc', 'n1_sars_cov2_below_lod', 'n2_sars_cov2_below_lod']
df_everything_else = df_pid171_v2[df_pid171_v2.columns[~df_pid171_v2.columns.isin(melt_clms)]]


df_complete = pd.merge(df_melt, df_everything_else, left_on = "sample_id",
                                                    right_on = "sample_id",
                                                   how = "left")


In [14]:
df_complete.shape

(1714, 97)

In [15]:
mismatch = compare_similar_dataframes(df_pid171, df_complete)

The shapes of Dataframe1 and Dataframe2 are identical: (1714, 97)

The columns of Dataframe1 and Dataframe2 are identical, congrats!

NOT IDENTICAL: c1_ww_analysis_reporting_complete

NOT IDENTICAL: c2_viral_lab_report_form_complete

NOT IDENTICAL: cbod

NOT IDENTICAL: cod

NOT IDENTICAL: collection_date

NOT IDENTICAL: collection_storage_temp

NOT IDENTICAL: collection_storage_time

NOT IDENTICAL: collection_water_temp

NOT IDENTICAL: concentration_method

NOT IDENTICAL: conductivity

NOT IDENTICAL: data_flag_notes

NOT IDENTICAL: e_coli

NOT IDENTICAL: e_sarbeco_sars_cov2_avg_conc

NOT IDENTICAL: e_sarbeco_sars_cov2_below_lod

NOT IDENTICAL: equiv_sewage_amt

NOT IDENTICAL: extraction_method

NOT IDENTICAL: fecal_coliform

NOT IDENTICAL: flow_rate

NOT IDENTICAL: hum_frac_chem_conc

NOT IDENTICAL: hum_frac_mic_conc

NOT IDENTICAL: hum_frac_mic_unit

NOT IDENTICAL: inhibition_adjust

NOT IDENTICAL: inhibition_detect

NOT IDENTICAL: inhibition_method

NOT IDENTICAL: label_date

NOT IDE

In [16]:
mismatch

['c1_ww_analysis_reporting_complete',
 'c2_viral_lab_report_form_complete',
 'cbod',
 'cod',
 'collection_date',
 'collection_storage_temp',
 'collection_storage_time',
 'collection_water_temp',
 'concentration_method',
 'conductivity',
 'data_flag_notes',
 'e_coli',
 'e_sarbeco_sars_cov2_avg_conc',
 'e_sarbeco_sars_cov2_below_lod',
 'equiv_sewage_amt',
 'extraction_method',
 'fecal_coliform',
 'flow_rate',
 'hum_frac_chem_conc',
 'hum_frac_mic_conc',
 'hum_frac_mic_unit',
 'inhibition_adjust',
 'inhibition_detect',
 'inhibition_method',
 'label_date',
 'label_printed',
 'latestrejectcomment',
 'lod_sewage',
 'micro_lab_id',
 'micro_lab_name',
 'micro_received',
 'n1andn2combined_sars_cov2_avg_conc',
 'n1andn2combined_sars_cov2_below_lod',
 'n3_sars_cov2_avg_conc',
 'n3_sars_cov2_below_lod',
 'n_sarbeco_sars_cov2_avg_conc',
 'n_sarbeco_sars_cov2_below_lod',
 'nh4_nh3',
 'niid_2019ncov_n_sars_cov2_avg_conc',
 'niid_2019ncov_n_sars_cov2_below_lod',
 'notes_ww',
 'ntc_amplify',
 'orf1ab_s

In [17]:
pwd

'C:\\AXG5303_Repos\\CDC_wastewater_to_REDCap'